<a href="https://colab.research.google.com/github/rbaner21/technosignature-pipeline-v2/blob/main/notebooks/3b_merge_tess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: ensure outputs/ exists & load base matrix + tess flags
import os, pandas as pd
os.makedirs("outputs", exist_ok=True)

base = pd.read_csv("outputs/feature_base.csv")
tess = pd.read_csv("inputs/tess_flags.csv")
print(f"Loaded base ({len(base)}) + tess_flags ({len(tess)})")


In [ ]:
# Cell 2: merge TESS into the base matrix
fm = base.merge(tess[["tic_id","tess_flag"]], on="tic_id", how="left")
fm["tess_flag"] = fm["tess_flag"].fillna(False)

fm.to_csv("outputs/feature_matrix.csv", index=False)
print(f"✅ Final feature_matrix.csv ({len(fm)} rows)")
